In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
file_path = r'/Users/judy/Downloads/UK House price index.xlsx'

In [16]:
file_path = '/Users/judy/Downloads/UK House price index.xlsx'  # update if your file is elsewhere
df = pd.read_excel(file_path, sheet_name='Average price', index_col=None)
df.head()

,Unnamed: 0,City of London,Barking & Dagenham,Barnet,Bexley,Brent,Bromley,Camden,Croydon,Ealing,...,NORTH WEST,YORKS & THE HUMBER,EAST MIDLANDS,WEST MIDLANDS,EAST OF ENGLAND,LONDON,SOUTH EAST,SOUTH WEST,Unnamed: 47,England
0,NaT,E09000001,E09000002,E09000003,E09000004,E09000005,E09000006,E09000007,E09000008,E09000009,...,E12000002,E12000003,E12000004,E12000005,E12000006,E12000007,E12000008,E12000009,NaN,E92000001
1,1995-01-01,90347,51870,98948,64956,76880,83082,119775,70118,85469,...,40907,42171,43856,46470,56098,79687,64502,52799,NaN,50231
2,1995-02-01,81213,52513,98848,64786,77651,83068,118365,69908,86551,...,40877,41912,44344,47249,55991,77913,64196,52462,NaN,50130
3,1995-03-01,78168,52701,97848,64366,77644,82856,119131,69666,87067,...,41351,42544,43701,47345,55574,79110,64597,51716,NaN,50229
4,1995-04-01,76172,54618,96273,64276,78668,82525,118948,69562,87933,...,41195,42934,44414,47359,55966,79708,65111,52877,NaN,50597


In [18]:
df = df.transpose()

In [20]:
df.columns = df.iloc[0]
df = df.drop(df.index[0])

/Users/judy/Desktop/anaconda3/lib/python3.12/site-packages/pandas/core/indexes/base.py:7631: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(index_like, name=name, copy=copy)


In [22]:
df = df.reset_index()
df = df.rename(columns={'index': 'Borough'})

In [24]:
melted = pd.melt(df, id_vars=['Borough'], var_name='Date', value_name='Average Price')

In [26]:
melted['Date'] = pd.to_datetime(melted['Date'], errors='coerce')
melted['Average Price'] = pd.to_numeric(melted['Average Price'], errors='coerce')

In [28]:
cleaned = melted.dropna()

In [30]:
cleaned['Year'] = cleaned['Date'].dt.year

/var/folders/8v/l804p1v56rg0kxpqswf28qtw0000gn/T/ipykernel_13768/2648091114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned['Year'] = cleaned['Date'].dt.year


In [32]:
cleaned.loc[:, 'Year'] = cleaned['Date'].dt.year

In [34]:
grouped = cleaned.groupby(['Borough', 'Year']).mean(numeric_only=True).reset_index()

In [36]:
df_1998 = grouped[grouped['Year'] == 1998]
df_2018 = grouped[grouped['Year'] == 2018]

In [38]:
merged = pd.merge(df_1998, df_2018, on='Borough', suffixes=('_1998', '_2018'))
merged['Growth'] = merged['Average Price_2018'] / merged['Average Price_1998']

In [40]:
top_boroughs = merged.sort_values('Growth', ascending=False)
print(top_boroughs[['Borough', 'Growth']].head(10))

                 Borough    Growth
14               Hackney  6.198334
41        Waltham Forest  5.835128
37             Southwark  5.516370
27              Lewisham  5.447926
43           Westminster  5.352731
31                Newham  5.307660
6         City of London  5.302624
21          Inner London  5.170824
16              Haringey  5.134551
23  Kensington & Chelsea  5.082174


In [42]:
#Hackney has seen the greatest increase 
